In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -q split-folders

In [ ]:
# #!/usr/bin/env python3
# # -*- coding: utf-8 -*-
# """
# Created on Thu Oct 19 19:22:28 2023

# @author: kmark7
# """

import splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow_hub as hub

splitfolders.ratio("drive/MyDrive/KutyAI/images", output="drive/MyDrive/KutyAI/dogs", seed=1337, ratio=(.6, .4), group_prefix=None, move=True)

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 256
train_dir = "drive/MyDrive/KutyAI/dogs/train/"
test_dir = "drive/MyDrive/KutyAI/dogs/val/"

train_datagen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=40,
    shear_range=0.3,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
print("Testing images:")
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

data_dir = pathlib.Path("drive/MyDrive/KutyAI/dogs/train/")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5"
resnet = hub.KerasLayer(resnet_url, trainable=False, name='resnet50', input_shape=IMAGE_SHAPE+(3,))
model = tf.keras.Sequential([
    resnet,
    layers.BatchNormalization(),
    layers.Dense(420, activation='softmax', name='output_layer')
])

model.summary()

class_weights = {}
for i in range(420):
    class_weights[i] = 1.0

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping = EarlyStopping(
    min_delta=0.01,
    patience=5,
    restore_best_weights=True,
)

train_datagen_augmented = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=40,
    shear_range=0.3,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

augmented_train_data = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

combined_train_data = tf.data.Dataset.from_generator(
    lambda: train_data,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 420), dtype=tf.float32)
    )
).concatenate(
    tf.data.Dataset.from_generator(
        lambda: augmented_train_data,
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 420), dtype=tf.float32)
        )
    )
)

history = model.fit(
    combined_train_data,
    epochs=15,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[early_stopping],
    class_weight=class_weights
)

def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    epochs = range(len(history.history['loss']))
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

plot_loss_curves(history)

model.save('KutyaJoModel')
model2 = tf.keras.models.load_model("KutyaJoModel")

def load_and_prep_image(filename, img_shape=224):
    img = tf.io.read_file(filename)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, size=[img_shape, img_shape])
    img = tf.expand_dims(img, axis=0)
    img = img / 255.
    return img

image = load_and_prep_image("contennt/drive/MyDrive/KutyAI/elsoTeszt.JPG")
pred = model2.predict(image)
pred_class = class_names[np.argmax(pred)]

print(pred_class)



Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:00,  3.94 files/s]
Copying files: 2 files [00:00,  2.54 files/s]
Copying files: 38 files [00:00, 65.19 files/s]
Copying files: 59 files [00:01, 55.69 files/s]
Copying files: 87 files [00:01, 87.88 files/s]
Copying files: 105 files [00:01, 65.53 files/s]
Copying files: 133 files [00:01, 93.98 files/s]
Copying files: 157 files [00:02, 74.81 files/s]
Copying files: 190 files [00:02, 106.66 files/s]
Copying files: 210 files [00:02, 78.68 files/s] 
Copying files: 240 files [00:03, 106.18 files/s]
Copying files: 260 files [00:03, 78.66 files/s] 
Copying files: 290 files [00:03, 105.70 files/s]
Copying files: 310 files [00:04, 78.68 files/s] 
Copying files: 340 files [00:04, 105.83 files/s]
Copying files: 360 files [00:04, 81.28 files/s] 
Copying files: 391 files [00:04, 110.02 files/s]
Copying files: 411 files [00:05, 80.22 files/s] 
Copying files: 441 files [00:05, 106.37 files/s]
Copying files: 460 files [00:05, 80.07 fi

Training images:
Found 30410 images belonging to 420 classes.
Testing images:
Found 11678 images belonging to 420 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (KerasLayer)       (None, 1001)              44677609  
                                                                 
 batch_normalization (Batch  (None, 1001)              4004      
 Normalization)                                                  
                                                                 
 output_layer (Dense)        (None, 420)               420840    
                                                                 
Total params: 45102453 (172.05 MB)
Trainable params: 422842 (1.61 MB)
Non-trainable params: 44679611 (170.44 MB)
_________________________________________________________________
Found 30410 images belonging to 420 classes.
Epoch 1/15
  1/119 [...................